risk free rate data: https://www.bis.org/statistics/cbpol.htm

call and put: https://medium.com/swlh/calculating-option-premiums-using-the-black-scholes-model-in-python-e9ed227afbee

<!-- Varaince of call and put: https://github.com/dedwards25/Python_Option_Pricing/blob/master/GBS.ipynb -->

In [1]:
# !wget https://raw.githubusercontent.com/pambnb/Option/main/data/EURCAD1440.csv
# !wget https://raw.githubusercontent.com/pambnb/Option/main/data/EURJPY1440.csv
# !wget https://raw.githubusercontent.com/pambnb/Option/main/data/EURUSD1440.csv
# !wget https://raw.githubusercontent.com/pambnb/Option/main/data/NZDUSD1440.csv
# !wget https://raw.githubusercontent.com/pambnb/Option/main/data/rfr.csv

--2021-05-18 15:33:22--  https://raw.githubusercontent.com/pambnb/Option/main/data/EURCAD1440.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 251869 (246K) [text/plain]
Saving to: ‘EURCAD1440.csv’

EURCAD1440.csv      100%[===================>] 245.97K   785KB/s    in 0.3s    

2021-05-18 15:33:24 (785 KB/s) - ‘EURCAD1440.csv’ saved [251869/251869]

--2021-05-18 15:33:24--  https://raw.githubusercontent.com/pambnb/Option/main/data/EURJPY1440.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252348 (246K) [text/plain]
Saving 

In [2]:
from math import log, sqrt, pi, exp
from scipy.stats import norm
from datetime import datetime, date
import numpy as np
import pandas as pd
from pandas import DataFrame
import math

In [3]:
df = pd.read_csv("rfr.csv") 

In [4]:
my_country = df["Reference area"].unique().tolist()
my_currency = ['ARS', 'AUD', 'BRL', 'CAD', 'CHF',
       'CLP', 'CNY', 'COP', 'CZK', 'DKK',
       'GBP', 'HKD', 'HRK', 'HUF',
       'IDR', 'ILS', 'INR', 'ISK', 'JPY', 'KRW',
       'MKD', 'MXN', 'RM', 'NOK', 'NZD',
       'PEN', 'PHP', 'PLN', 'RON', 'RSD', 'RUB',
       'SAR', 'SEK', 'THB', 'TRY', 'USD',
       'EUR', 'ZAR']
my_dict = dict(zip(my_country, my_currency))
df["Reference area"] = df["Reference area"].replace(my_dict)
df = df.rename(columns={"Reference area": "Currency abbreviation"})
df = df.drop(['FREQ', "Frequency", "REF_AREA", "Time Period"], axis = 1)
# df = df.set_index(df.iloc[:, 0].name)
df.head()

,Currency abbreviation,1946-01-01,1946-01-02,1946-01-03,1946-01-04,1946-01-05,1946-01-06,1946-01-07,1946-01-08,1946-01-09,...,2021-03-20,2021-03-21,2021-03-22,2021-03-23,2021-03-24,2021-03-25,2021-03-26,2021-03-27,2021-03-28,2021-03-29
0,ARS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.0,38.0,38.00,38.00,38.00,NaN,NaN,NaN,NaN,NaN
1,AUD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.10,0.10,0.10,NaN,NaN,NaN,NaN,NaN
2,BRL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CAD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.25,0.25,0.25,0.25,0.25,NaN,NaN,NaN
4,CHF,1.5,1.5,1.5,1.5,NaN,NaN,1.5,1.5,1.5,...,NaN,NaN,-0.75,-0.75,-0.75,-0.75,-0.75,NaN,NaN,-0.75


In [5]:
df[df.columns[1:]] = df[df.columns[1:]].mask(df[df.columns[1:]].eq(-1)).ffill(axis=1)
# df.loc[df["Currency abbreviation"]== "EUR", '2007-01-01': '2021-03-29']
df.loc[:, '2007-01-01':'2021-03-29']

,2007-01-01,2007-01-02,2007-01-03,2007-01-04,2007-01-05,2007-01-06,2007-01-07,2007-01-08,2007-01-09,2007-01-10,...,2021-03-20,2021-03-21,2021-03-22,2021-03-23,2021-03-24,2021-03-25,2021-03-26,2021-03-27,2021-03-28,2021-03-29
0,6.250,6.2500,6.2500,6.500,6.5000,6.5000,6.5000,6.5000,6.5000,6.5000,...,38.000,38.000,38.000,38.000,38.000,38.000,38.000,38.000,38.000,38.000
1,6.250,6.2500,6.2500,6.250,6.2500,6.2500,6.2500,6.2500,6.2500,6.2500,...,0.100,0.100,0.100,0.100,0.100,0.100,0.100,0.100,0.100,0.100
2,13.250,13.2500,13.2500,13.250,13.2500,13.2500,13.2500,13.2500,13.2500,13.2500,...,2.750,2.750,2.750,2.750,2.750,2.750,2.750,2.750,2.750,2.750
3,4.250,4.2500,4.2500,4.250,4.2500,4.2500,4.2500,4.2500,4.2500,4.2500,...,0.250,0.250,0.250,0.250,0.250,0.250,0.250,0.250,0.250,0.250
4,2.000,2.0000,2.0000,2.000,2.0000,2.0000,2.0000,2.0000,2.0000,2.0000,...,-0.750,-0.750,-0.750,-0.750,-0.750,-0.750,-0.750,-0.750,-0.750,-0.750
5,5.250,5.2500,5.2500,5.250,5.2500,5.2500,5.2500,5.2500,5.2500,5.2500,...,0.500,0.500,0.500,0.500,0.500,0.500,0.500,0.500,0.500,0.500
6,6.120,6.1200,6.1200,6.120,6.1200,6.1200,6.1200,6.1200,6.1200,6.1200,...,3.850,3.850,3.850,3.850,3.850,3.850,3.850,3.850,3.850,3.850
7,7.500,7.5000,7.5000,7.500,7.5000,7.5000,7.5000,7.5000,7.5000,7.5000,...,1.750,1.750,1.750,1.750,1.750,1.750,1.750,1.750,1.750,1.750
8,2.500,2.5000,2.5000,2.500,2.5000,2.5000,2.5000,2.5000,2.5000,2.5000,...,0.250,0.250,0.250,0.250,0.250,0.250,0.250,0.250,0.250,0.250
9,3.750,3.7500,3.7500,3.750,3.7500,3.7500,3.7500,3.7500,3.7500,3.7500,...,-0.500,-0.500,-0.500,-0.500,-0.500,-0.500,-0.500,-0.500,-0.500,-0.500


In [6]:
df.loc[:, '2007-01-01':'2021-03-29'].isnull().values.any()

False

In [7]:
def get_risk_free_rate(date, curr):
    return float(df.loc[df["Currency abbreviation"]== curr][date].values)
# https://datatofish.com/select-rows-pandas-dataframe/

In [8]:
# S = spot price of an asset
# K = strike price
# r = risk-free interest rate
# T = time to maturity
# sigma = volatility of the asset

def d1(S,K,T,r,sigma):
    return(log(S/K)+(r+sigma**2/2.)*T)/sigma*sqrt(T)

def d2(S,K,T,r,sigma):
    return d1(S,K,T,r,sigma)-sigma*sqrt(T)

def bs_call(S,K,T,r,sigma):
    return S*norm.cdf(d1(S,K,T,r,sigma))-K*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma))
  
def bs_put(S,K,T,r,sigma):
    return K*exp(-r*T)-S+bs_call(S,K,T,r,sigma)

In [9]:
def d1_var(S,K,T,r,rf,sigma):
    return(log(S/K)+(r-rf+sigma**2/2.)*T)/sigma*sqrt(T)

def d2_var(S,K,T,r,rf,sigma):
    return d1(S,K,T,r,rf,sigma)-sigma*sqrt(T)

def bs_call_var(S,K,T,r,rf,sigma):
    return S*exp(-rf*T)*norm.cdf(d1(S,K,T,r,sigma))-K*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma))
  
def bs_put_var(S,K,T,r,rf,sigma):
    return K*exp(-r*T)*norm.cdf(-d2(S,K,T,r,sigma))-S*exp(-rf*T)*norm.cdf(-d1(S,K,T,r,sigma))

In [10]:
from datetime import date, timedelta
# start_date = date(2013, 1, 1) #year month date
# end_date = date(2015, 6, 2)
def my_date(start_date, end_date, step):
    delta = timedelta(days=step)
    list_date = []
    while start_date <= end_date:
        start_date += delta
        list_date.append(start_date.strftime("%Y-%m-%d"))
    return list_date

In [11]:
# check function date and risk free rate
start_date = date(2019, 1, 1)
end_date = date(2020, 1, 1)
# my_date(start_date, end_date)[0]
loop_date = my_date(start_date, end_date, 1)[0]
r = get_risk_free_rate(loop_date, "JPY")
print(loop_date, r)

2019-01-02 -0.1


In [12]:
def open_file(qq):
    data_list = []
    name = str(qq) + '.csv'
    with open(name, 'r') as reading:
        file_input = reading.read().split('\n')

    for row in file_input:
        data_list.append(row.split('\t'))
    data = pd.DataFrame(data_list,columns=['date', 'open', 'high', 'low', 'close', 'vol'])
    data['close'] = pd.to_numeric(data['close'],errors = 'coerce')
    return data

In [13]:
# check open file and function date
dr = open_file("EURUSD1440")
a_date = my_date(start_date, end_date, 1)[0]+str(' 00:00')
dr.loc[dr['date']== a_date]

,date,open,high,low,close,vol
3756,2019-01-02 00:00,1.14612,1.14967,1.13092,1.13121,455432


In [14]:
# check variance
V = 100
S = V * open_file("EURUSD1440")['close'][0]
#float(EURUSD1['close'][30*3])
alpha = 0.9
K = S*(1+alpha)
T = 3*30
r = get_risk_free_rate("2007-01-01", "EUR")
rf = get_risk_free_rate("2007-01-01", "USD")
sigma = open_file("EURUSD1440")[0:30]['close'].var()
EURUSD_var = bs_call_var(S,K,T,r,rf,sigma)

In [15]:
# PAM 9 may 

# sigma
open_file("EURUSD1440")[0:30]['close']

0     1.32120
1     1.32724
2     1.31690
3     1.30860
4     1.29995
5     1.30039
6     1.30356
7     1.29919
8     1.29347
9     1.28910
10    1.29212
11    1.29242
12    1.29299
13    1.29165
14    1.29368
15    1.29590
16    1.29536
17    1.29590
18    1.29466
19    1.30200
20    1.29609
21    1.29357
22    1.29144
23    1.29192
24    1.29601
25    1.29617
26    1.30175
27    1.30123
28    1.29562
29    1.29582
Name: close, dtype: float64

In [16]:
def call_option_black(data,alpha,V):
    ''' 
    data = file name (str name) ex open_file("EURUSD1440")
    S = current spot rate = V * currency
    K = strike price = S * (1+alpha)
    r = domestic risk free simple interest rate
    T = time to maturity (assume 90 days or 3 months)
    sigma =  volatility of the FX rate of previous T days (get from Garman-Kohlhagen model)
    '''
    T = 3*30
    start_date = date(2020, 1, 1) # date(2007, 1, 1) is first date of this period
    end_date = date(2021, 3, 1)
    length_date = len(my_date(start_date, end_date, 1))
    domestic_currency = [data[i:i+3] for i in range(0, len(data), 3)][0]
    foreign_currency = [data[i:i+3] for i in range(0, len(data), 3)][1]
    list_var_call = []
    for i in range(0,length_date):
        S = V * open_file(data)['close'][i] # fx folder
        K = S*(1+alpha)
        loop_date_1 = my_date(start_date, end_date, 1)[i] # step  = 1
        r = get_risk_free_rate(loop_date_1, domestic_currency) # rfr folder
        P = np.log((open_file(data).loc[i+1]['close']+1e-5)/(open_file(data).loc[i]['close']+1e-6))
        sigma = math.sqrt(1/(30+1)*np.sum(P**2)-1/(30*(30-1))*(np.sum(P)**2))
        var_black =  bs_call(S,K,T,r,sigma)
        list_var_call.append(var_black)
    return list_var_call
        

In [17]:
def put_option_black(data,alpha,V):
    ''' 
    data = file name (str name) ex open_file("EURUSD1440")
    S = current spot rate = V * currency
    K = strike price = S * (1+alpha)
    r = domestic risk free simple interest rate
    T = time to maturity (assume 90 days or 3 months)
    sigma =  volatility of the FX rate of previous T days (get from Garman-Kohlhagen model)
    '''
    T = 3*30
    start_date = date(2020, 1, 1)
    end_date = date(2021, 3, 1)
    length_date = len(my_date(start_date, end_date, 1))
    domestic_currency = [data[i:i+3] for i in range(0, len(data), 3)][0]
    foreign_currency = [data[i:i+3] for i in range(0, len(data), 3)][1]
    list_var_call = []
    for i in range(0,length_date):
        S = V * open_file(data)['close'][i] # fx folder
        K = S*(1+alpha)
        loop_date_1 = my_date(start_date, end_date, 1)[i] # step  = 1
        r = get_risk_free_rate(loop_date_1, domestic_currency) # rfr folder
        P = np.log((open_file(data).loc[i+1]['close']+1e-5)/(open_file(data).loc[i]['close']+1e-6))
        sigma = math.sqrt(1/(30+1)*np.sum(P**2)-1/(30*(30-1))*(np.sum(P)**2))
        var_black =  bs_put(S,K,T,r,sigma)
        list_var_call.append(var_black)
    return list_var_call
        

In [18]:
def call_option_garman(data,alpha,V):
    ''' 
    data = file name (str name) ex open_file("EURUSD1440")
    S = current spot rate = V * currency
    K = strike price = S * (1+alpha)
    r = domestic risk free simple interest rate
    rf = foreign risk free simple interest rate
    T = time to maturity (assume 90 days or 3 months)
    sigma =  volatility of the FX rate of previous T days (get from Garman-Kohlhagen model)
    '''
    T = 3*30
    start_date = date(2020, 1, 1)
    end_date = date(2021, 3, 1)
    length_date = len(my_date(start_date, end_date, 1))
    domestic_currency = [data[i:i+3] for i in range(0, len(data), 3)][0]
    foreign_currency = [data[i:i+3] for i in range(0, len(data), 3)][1]
    list_var_call = []
    for i in range(0,length_date):
        S = V * open_file(data)['close'][i] # fx folder
        K = S*(1+alpha)
        loop_date_1 = my_date(start_date, end_date, 1)[i] # step  = 1
        r = get_risk_free_rate(loop_date_1, domestic_currency) # rfr folder
        rf = get_risk_free_rate(loop_date_1, foreign_currency) # rfr folder
        P = np.log((open_file(data).loc[i+1]['close']+1e-5)/(open_file(data).loc[i]['close']+1e-6))
        sigma = math.sqrt(1/(30+1)*np.sum(P**2)-1/(30*(30-1))*(np.sum(P)**2))
        var_garman =  bs_call_var(S,K,T,r,rf,sigma)
        list_var_call.append(var_garman)
    return list_var_call
        

In [19]:
def put_option_garman(data,alpha,V):
    ''' 
    data = file name (str name) ex open_file("EURUSD1440")
    S = current spot rate = V * currency
    K = strike price = S * (1+alpha)
    r = domestic risk free simple interest rate
    rf = foreign risk free simple interest rate
    T = time to maturity (assume 90 days or 3 months)
    sigma =  volatility of the FX rate of previous T days (get from Garman-Kohlhagen model)
    '''
    T = 3*30
    start_date = date(2020, 1, 1)
    end_date = date(2021, 3, 1)
    length_date = len(my_date(start_date, end_date, 1))
    domestic_currency = [data[i:i+3] for i in range(0, len(data), 3)][0]
    foreign_currency = [data[i:i+3] for i in range(0, len(data), 3)][1]
    list_var_call = []
    for i in range(0,length_date):
        S = V * open_file(data)['close'][i] # fx folder
        K = S*(1+alpha)
        loop_date_1 = my_date(start_date, end_date, 1)[i] # step  = 1
        r = get_risk_free_rate(loop_date_1, domestic_currency) # rfr folder
        rf = get_risk_free_rate(loop_date_1, foreign_currency) # rfr folder
        P = np.log((open_file(data).loc[i+1]['close']+1e-5)/(open_file(data).loc[i]['close']+1e-6))
        sigma = math.sqrt(1/(30+1)*np.sum(P**2)-1/(30*(30-1))*(np.sum(P)**2))
        var_garman =  bs_put_var(S,K,T,r,rf,sigma)
        list_var_call.append(var_garman)
    return list_var_call
        

In [20]:
data =  'NZDUSD1440'
alpha = 1.4
V = 1000
call_option_black(data,alpha,V)

[705.04,
 705.97,
 705.6700000000001,
 695.17,
 686.57,
 686.14,
 690.9,
 691.79,
 687.0500000000001,
 688.1899999999999,
 690.1500000000001,
 691.88,
 695.97,
 689.4,
 692.0699999999999,
 694.68,
 694.94,
 694.23,
 698.87,
 700.27,
 699.6899999999999,
 696.3199999999999,
 695.3600000000001,
 695.26,
 695.73,
 686.9,
 689.21,
 682.27,
 680.84,
 683.47,
 684.2600000000001,
 680.21,
 687.5600000000001,
 684.4200000000001,
 682.9300000000001,
 683.4200000000001,
 682.98,
 689.1899999999999,
 692.46,
 692.5600000000001,
 696.9499999999999,
 699.7600000000001,
 699.9100000000001,
 701.3199999999999,
 705.6700000000001,
 704.02,
 706.05,
 708.1800000000001,
 709.98,
 696.9499999999999,
 700.62,
 693.38,
 687.54,
 684.47,
 672.95,
 684.3199999999999,
 677.1,
 687.3000000000001,
 690.0500000000001,
 693.6399999999999,
 695.92,
 684.83,
 693.5,
 695.5,
 696.76,
 696.4100000000001,
 703.23,
 706.36,
 709.65,
 711.02,
 710.75,
 710.15,
 716.0999999999999,
 716.5500000000001,
 711.0,
 713.43999971